# Ch 15. BERT 사전학습모형에 대한 미세조정학습

## 15.1 BERT 학습을 위한 전처리



In [1]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence1 = "What a beautiful day!"
sentence2 = "Nvidia Titan XP has 12GB of VRAM"

# 1. 토큰화 결과
print(sentence1, '토큰화 결과:', tokenizer.tokenize(sentence1))
print(sentence2, '토큰화 결과:', tokenizer.tokenize(sentence2))

What a beautiful day! 토큰화 결과: ['what', 'a', 'beautiful', 'day', '!']
Nvidia Titan XP has 12GB of VRAM 토큰화 결과: ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [2]:
# 2. BERT 모형 입력 생성
inputs = tokenizer([sentence1, sentence2], padding=True)
print('BERT 입력:', inputs)

BERT 입력: {'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [3]:
# 3. 두 문장으로 이루어진 시퀀스에 대한 BERT 모형 입력 생성
inputs = tokenizer(sentence1, sentence2, padding=True)
print('두 문장 시퀀스에 대한 BERT 입력:', inputs)

두 문장 시퀀스에 대한 BERT 입력: {'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


## 15.2 트랜스포머의 트레이너를 이용한 미세조정학습


In [5]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\wousi\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count:  1600
Test set count:  400


In [6]:
from transformers import BertTokenizerFast, BertForSequenceClassification 

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors="pt")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

(1) https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html

In [7]:
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = OurDataset(train_input, y_train)
test_dataset = OurDataset(test_input, y_test)

In [8]:
# !pip install datasets

#https://huggingface.co/docs/datasets/

In [9]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

(1) https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.TrainingArguments

(2) https://huggingface.co/transformers/main_classes/trainer.html?highlight=trainingarguments#transformers.Trainer

In [10]:
from transformers import Trainer, TrainingArguments

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir='./results',          # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs=2,              # 학습 에포크 수
    per_device_train_batch_size=8,   # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size=16,   # 평가에 사용할 배치 사이즈
)

trainer = Trainer(
    model=model,                     # 학습할 모형
    args=training_args,              # 위에서 정의한 학습 매개변수
    train_dataset=train_dataset,     # 훈련 데이터셋
    compute_metrics=compute_metrics,
)

# 미세조정학습 실행
trainer.train()

C:\Users\wousi\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 400
<ipython-input-7-07e699bc1a98>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=400, training_loss=0.5342620849609375, metrics={'train_runtime': 7953.719, 'train_samples_per_second': 0.402, 'train_steps_per_second': 0.05, 'total_flos': 841955377152000.0, 'train_loss': 0.5342620849609375, 'epoch': 2.0})

In [11]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 400
  Batch size = 16
<ipython-input-7-07e699bc1a98>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


{'eval_loss': 0.3477444052696228,
 'eval_accuracy': 0.8675,
 'eval_runtime': 521.0927,
 'eval_samples_per_second': 0.768,
 'eval_steps_per_second': 0.048,
 'epoch': 2.0}

## 15.3 파이토치를 이용한 미세조정학습



In [12]:
del model
del trainer
torch.cuda.empty_cache()

In [13]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)

In [14]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\wousi/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin fr

In [15]:
# BERT를 포함한 신경망 모형
class MyModel(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels): 
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model

        # 분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)

    def forward(self, inputs):
        # BERT 모형에 입력을 넣고 출력을 받음
        outputs = self.pretrained_model(**inputs)
        # BERT 출력에서 CLS 토큰에 해당하는 부분만 가져옴
        bert_clf_token = outputs.last_hidden_state[:,0,:]
        
        return self.classifier(bert_clf_token)

# token_size는 BERT 토큰과 동일, bert_model.config.hidden_size로 알 수 있음
model = MyModel(bert_model, num_labels=2, token_size=bert_model.config.hidden_size) 

In [16]:
from transformers import AdamW
import torch.nn.functional as F
import time

# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)  # 모형을 GPU로 복사
model.train()     # 학습모드로 전환

optim = AdamW(model.parameters(), lr=5e-5) # 옵티마이저를 트랜스포머가 제공하는 AdamW로 설정
criterion = torch.nn.CrossEntropyLoss()    # 멀티클래스이므로 크로스 엔트로피를 손실함수로 사용

start = time.time() # 시작시간 기록
num_epochs = 4      # 학습 epoch를 4회로 설정
for epoch in range(num_epochs):
    total_epoch_loss = 0  # epoch의 총 loss 초기화
    for step, batch in enumerate(train_loader):
        optim.zero_grad()     # 그래디언트 초기화
        # 배치에서 label을 제외한 입력만 추출하여  GPU로 복사
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'} 
        labels = batch['labels'].to(device) # 배치에서 라벨을 추출하여 GPU로 복사
        outputs = model(inputs) # 모형으로 결과 예측
        # 두 클래스에 대해 예측하고 각각 비교해야 하므로 labels에 대해 원핫인코딩을 적용한 후에 손실을 게산
        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float()) # loss 계산

        if (step+1) % 100 == 0: # 100 배치마다 경과한 시간과 loss를 출력
            elapsed = time.time() - start
            print('Epoch %d, batch %d, elapsed time: %.2f, loss: %.4f' % (epoch+1, step+1, elapsed, loss))
        total_epoch_loss += loss
        loss.backward() # 그래디언트 계산
        optim.step()    # 가중치 업데이트
    avg_epoch_loss = total_epoch_loss / len(train_loader) # epoch의 평균 loss 계산
    print('Average loss for epoch %d: %.4f' % (epoch+1, avg_epoch_loss))


C:\Users\wousi\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-7-07e699bc1a98>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


Epoch 1, batch 100, elapsed time: 3045.66, loss: 0.8186
Epoch 1, batch 200, elapsed time: 6404.75, loss: 0.2309
Average loss for epoch 1: 0.5688
Epoch 2, batch 100, elapsed time: 9363.59, loss: 0.0715
Epoch 2, batch 200, elapsed time: 12612.83, loss: 1.0242
Average loss for epoch 2: 0.2949
Epoch 3, batch 100, elapsed time: 15721.37, loss: 0.1780
Epoch 3, batch 200, elapsed time: 18726.02, loss: 0.0883
Average loss for epoch 3: 0.1288
Epoch 4, batch 100, elapsed time: 21985.28, loss: 0.0200
Epoch 4, batch 200, elapsed time: 25027.74, loss: 0.0016
Average loss for epoch 4: 0.0856


In [17]:
from datasets import load_metric

test_loader = DataLoader(test_dataset, batch_size=16)

metric= load_metric("accuracy")
model.eval()
for batch in test_loader:
    inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    
    with torch.no_grad(): # 학습할 필요가 없으므로 그래디언트 계산을 끔
        outputs = model(inputs)
        print(outputs) 
        '''tensor([[-0.8060,  0.5903],
        [-2.1077,  2.0186],
        [ 3.6877, -4.3673],
        [-0.2765,  0.0720],
        [-1.0088,  0.7596],
        [ 0.4830, -0.7912],
        [-2.8845,  2.9227],
        [-2.8038,  2.8391],
        [ 1.1291, -1.6503],
        [-2.0461,  1.9383],
        [ 2.7008, -3.3430],
        [ 2.9812, -3.6259],
        [ 0.0702, -0.3206],
        [-1.0552,  0.9338],
        [ 0.8283, -1.4272],
        [ 1.9559, -2.5911]])'''

    predictions = torch.argmax(outputs, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

metric.compute()

<ipython-input-7-07e699bc1a98>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}


tensor([[-0.8060,  0.5903],
        [-2.1077,  2.0186],
        [ 3.6877, -4.3673],
        [-0.2765,  0.0720],
        [-1.0088,  0.7596],
        [ 0.4830, -0.7912],
        [-2.8845,  2.9227],
        [-2.8038,  2.8391],
        [ 1.1291, -1.6503],
        [-2.0461,  1.9383],
        [ 2.7008, -3.3430],
        [ 2.9812, -3.6259],
        [ 0.0702, -0.3206],
        [-1.0552,  0.9338],
        [ 0.8283, -1.4272],
        [ 1.9559, -2.5911]])
tensor([[-1.9716,  1.9766],
        [-0.4418,  0.2468],
        [ 3.6586, -4.3835],
        [-0.1325, -0.2333],
        [ 0.1522, -0.3902],
        [-2.1822,  2.0739],
        [ 3.2508, -3.8518],
        [ 0.7327, -1.1931],
        [ 3.8036, -4.5063],
        [ 1.0416, -1.4032],
        [ 3.3240, -4.0591],
        [ 3.6364, -4.3915],
        [-2.7377,  2.6401],
        [ 3.3745, -4.2137],
        [ 3.4807, -4.1159],
        [ 3.6225, -4.3124]])
tensor([[ 2.4807, -3.1900],
        [-2.4452,  2.4336],
        [-2.2485,  2.1917],
        [-3.2669, 

tensor([[ 1.2210, -1.7425],
        [-0.3941,  0.0589],
        [ 3.6424, -4.3439],
        [ 3.4743, -4.1509],
        [ 0.9836, -1.3894],
        [ 3.7595, -4.5850],
        [ 3.5458, -4.2927],
        [ 3.6938, -4.4227],
        [-2.2859,  2.2903],
        [ 3.7387, -4.4860],
        [ 3.2054, -3.9213],
        [ 3.0346, -3.9262],
        [ 3.3050, -3.9874],
        [-3.5245,  3.6268],
        [ 2.1460, -2.6088],
        [ 0.2260, -0.4483]])
tensor([[-1.6892,  1.5327],
        [ 3.8062, -4.5481],
        [-3.4075,  3.5133],
        [ 1.1132, -1.4624],
        [-3.2044,  3.2578],
        [ 3.6009, -4.2945],
        [ 3.7115, -4.4614],
        [ 3.6774, -4.5079],
        [-2.6479,  2.6248],
        [ 2.9681, -3.5489],
        [ 2.4599, -2.9853],
        [ 3.2254, -3.9089],
        [ 3.4197, -4.1572],
        [ 3.1475, -3.6913],
        [ 3.5512, -4.3446],
        [ 3.6928, -4.5587]])
tensor([[ 2.3925, -2.8636],
        [-3.0669,  3.1507],
        [ 2.2679, -2.8092],
        [ 3.6593, 

{'accuracy': 0.8275}